# Data preparation

## Read csv file

In [ ]:
import pandas as pd

df = pd.read_csv('assignment/loan_data_set.csv')

## Encode labels

In [ ]:
df.replace('Male', 0, inplace=True)
df.replace('Female', 1, inplace=True)
df.replace('No', 0, inplace=True)
df.replace('Yes', 1, inplace=True)
df.replace('Not Graduate', 0, inplace=True)
df.replace('Graduate', 1, inplace=True)
df.replace('N', 0, inplace=True)
df.replace('Y', 0, inplace=True)
print(df)

## Fill empty cells

In [ ]:
df['LoanAmount'].fillna(df['LoanAmount'].mean(skipna=True), inplace=True)
print(df)